MNIST HANDWRITTING RECOGNITION

IMPORTING LIBRARIES AND LOADING THE DATATSET

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import keras
from tensorflow.keras import Sequential
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('mnist.h5')
print("Saving the model as mnist.h5")





Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 45s 758us/step - loss: 0.3601 - accuracy: 0.8887 - val_loss: 0.0761 - val_accuracy: 0.9779
Epoch 2/10
60000/60000 [==============================] - 32s 532us/step - loss: 0.1027 - accuracy: 0.9722 - val_loss: 0.0379 - val_accuracy: 0.9875
Epoch 3/10
60000/60000 [==============================] - 38s 631us/step - loss: 0.0761 - accuracy: 0.9803 - val_loss: 0.0354 - val_accuracy: 0.9892
Epoch 4/10
60000/60000 [==============================] - 31s 521us/step - loss: 0.0618 - accuracy: 0.9844 - val_loss: 0.0364 - val_accuracy: 0.9891
Epoch 5/10
60000/60000 [==============================] - 31s 516us/step - loss: 0.0503 - accuracy: 0.9872 - val_loss: 0.0282 - val_accuracy: 0.9909
Epoch 6/10
60000/60000 [==============================] - 31s 515us/step - loss: 0.0448 - accuracy: 0.9883 - val_loss: 0.0293 - val_accuracy: 0.9913
Epoch 7/10
60000/60000 [==============================] 

CREATE GUI TO PREDICT DIGITS

In [10]:
pip install pysimplegui

Note: you may need to restart the kernel to use updated packages.


GUI DIGIT RECOGNIZER

In [24]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import os
from PIL import ImageGrab, Image
import numpy as np

In [27]:
model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        #rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
       # HWND=self.canvas.find_withtag('draw_canvas')
        print(HWND)
       # rect = self.canvas.bbox(HWND)
        left= self.canvas.winfo_rootx()
        top=self.canvas.winfo_rooty()
        windowHeight=self.canvas.winfo_screenheight()
        windowWidth=self.canvas.winfo_screenwidth()
        width=self.canvas.winfo_width()
        height=self.canvas.winfo_height()
        right=windowWidth-(left + width)
        bottom=windowHeight-(top+height)
        rect=(left,top,right,bottom)
        print(rect)
        im = ImageGrab.grab(bbox=(left+10,top+40,right-500,bottom+100))
        im.show()

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
       
app = App()
mainloop()



140459061547024
(0, 47, 1134, 547)
